# Importación de librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import time

from time import time

%matplotlib inline
%matplotlib notebook
plt.style.use('default')

In [2]:
datos = pd.read_csv("events_up_to_01062018.csv",low_memory=False)


Se crea la columna quincena, para poder obtener features por quincena del año.

In [3]:
datos['timestamp'] = pd.to_datetime(datos['timestamp'])
dia = datos['timestamp'].dt.day
mes = datos['timestamp'].dt.month

quincenas = [False,0,2,4,6,8,10]

mes_num_q = mes.apply(lambda x: quincenas[x])

dia = dia.apply(lambda x: 0 if x<16 else 1)

datos['quincena'] = dia + mes_num_q

Durante el uso de logueos (new_vs_returning), se encontraron falta de fechas de registros de los usuarios.

In [4]:
personas = datos[['person','new_vs_returning']]
personas = personas.loc[personas['new_vs_returning'] == 'New']

In [5]:
usuariosyregistros = datos[['person','timestamp']].sort_values(by=['person','timestamp'],ascending = True)\
                                                .drop_duplicates('person').merge(personas,on='person',how='left')                                                                                
print('Sin fecha registro : ',usuariosyregistros.new_vs_returning.isnull().sum())
usuariosyregistros.fillna('New',inplace=True)

Sin fecha registro :  587


Se utiliza el primer evento según timestamp como nueva fecha de registro.

In [6]:
datos = pd.merge(datos,usuariosyregistros,how='left',on=['person','timestamp'])
print('Sin registro: ',datos[['person','new_vs_returning_y']].sort_values(by='new_vs_returning_y')\
                                            .drop_duplicates('person').new_vs_returning_y.isnull().sum())
datos = datos.rename(columns={'new_vs_returning_y':'new_vs_returning'}).drop(columns='new_vs_returning_x')

Sin registro:  0


# Operaciones con el dataframe

### 1: Cantidad de cada evento de cada persona

In [7]:
eventosPersona = datos[['person','event','timestamp']]
eventosPersona = eventosPersona.groupby(['person','event']).agg({'timestamp':'count'})
eventosPersona = eventosPersona.unstack(-1)
eventosPersona.columns = eventosPersona.columns.droplevel(0)
eventosPersona = eventosPersona.reset_index()
eventosPersona.fillna(0,inplace=True)

Mismos features pero por quincena:

In [8]:
eventosPersonaQ = datos[['person','event','quincena','timestamp']]
eventosPersonaQ = eventosPersonaQ.groupby(['person','event','quincena']).agg({'timestamp':'count'})
eventosPersonaQ = eventosPersonaQ.unstack('event')
eventosPersonaQ.columns = eventosPersonaQ.columns.droplevel(0)
eventosPersonaQ = eventosPersonaQ.unstack(-1)
eventosPersonaQ.head()

event    ad campaign hit                                          ...   \
quincena               0   1   2   3   4    5   6   7    8     9  ...    
person                                                            ...    
0008ed71             NaN NaN NaN NaN NaN  NaN NaN NaN  NaN   NaN  ...    
00091926             NaN NaN NaN NaN NaN  NaN NaN NaN  4.0  11.0  ...    
00091a7a             NaN NaN NaN NaN NaN  1.0 NaN NaN  NaN   NaN  ...    
000ba417             NaN NaN NaN NaN NaN  NaN NaN NaN  NaN   1.0  ...    
000c79fe             NaN NaN NaN NaN NaN  NaN NaN NaN  NaN   1.0  ...    

event    visited site                                           
quincena            0   1   2   3   4    5   6   7     8     9  
person                                                          
0008ed71          NaN NaN NaN NaN NaN  NaN NaN NaN   NaN   2.0  
00091926          NaN NaN NaN NaN NaN  NaN NaN NaN  18.0  16.0  
00091a7a          NaN NaN NaN NaN NaN  1.0 NaN NaN   NaN   NaN  
000ba417          NaN NaN NaN NaN NaN  NaN NaN NaN   NaN   6.0  
000c79fe          NaN NaN NaN NaN NaN  NaN NaN NaN   NaN   1.0  

[5 rows x 110 columns]

Se recorre eventosPersonaQ[i] adecuando los nombres de las columnas y mergeando en aux.

Por ej. eventosPersonQ['ad campaign hit'] se realiza un rename de las 10 columnas a 'ad_campaign_hit_'+str(j), siendo j del 0 al 9.

In [9]:
columnas = {}
aux = datos[['person','timestamp']].drop_duplicates('person')
for i in datos.event.unique():
    for j in range(0,10):
        columnas[j] = i+'_'+str(j)
    aux = pd.merge(aux,eventosPersonaQ[i],on='person',how='left')
    aux = aux.rename(columns=columnas)
aux.drop(columns='timestamp',inplace=True)
aux.fillna(0,inplace=True)
eventosPersonaQ = aux
eventosPersonaQ.head()

,person,viewed product_0,viewed product_1,viewed product_2,viewed product_3,viewed product_4,viewed product_5,viewed product_6,viewed product_7,viewed product_8,...,lead_0,lead_1,lead_2,lead_3,lead_4,lead_5,lead_6,lead_7,lead_8,lead_9
0,4886f805,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ad93850f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0297fc1e,13.0,64.0,12.0,21.0,34.0,27.0,28.0,72.0,37.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,2d681dd8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,cccea85e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,336.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 2: Nacionalidad de las personas

#### Se agrega el feature country cómo 1 si es brasilero y 0 si es de otro país (o Unknown). También el feature

In [10]:
compraron = datos[['person','event']].loc[datos['event']=='conversion']
compraron['convirtio'] = 1
compraron = compraron.drop(columns='event').drop_duplicates('person')
compraron.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4293 entries, 195 to 1759834
Data columns (total 2 columns):
person       4293 non-null object
convirtio    4293 non-null int64
dtypes: int64(1), object(1)
memory usage: 100.6+ KB


In [11]:
personas_paises = datos[['person','country']].dropna(axis=0).drop_duplicates('person')

¿De dónde son las personas que compraron?

In [12]:
personas_paises = pd.merge(personas_paises,compraron,on='person',how='left').fillna(0)
personas_paises.loc[personas_paises['convirtio']].country.value_counts()

Brazil    38242
Name: country, dtype: int64

In [13]:
personas_paises.drop(columns='convirtio',inplace=True)
personas_paises = datos[['person']].drop_duplicates('person').merge(personas_paises,how='left',on='person')

In [14]:
personas_paises.isnull().sum()

person       0
country    587
dtype: int64

In [15]:
personas_paises['country'].value_counts()

Brazil                   36802
Unknown                   1043
United States              260
Argentina                   31
Canada                      14
Mozambique                   8
France                       7
Uruguay                      7
Netherlands                  5
Bolivia                      5
Mexico                       4
Portugal                     4
Italy                        4
Paraguay                     4
Angola                       3
Spain                        3
Vietnam                      3
Bangladesh                   3
Dominican Republic           2
Israel                       2
Germany                      2
South Africa                 2
United Kingdom               2
Singapore                    2
Belgium                      2
Morocco                      2
São Tomé and Príncipe        1
Algeria                      1
Romania                      1
Guadeloupe                   1
India                        1
Bulgaria                     1
Burundi 

In [16]:
personas_paises.loc[personas_paises['country'] == 'Brazil','country'] = 1
personas_paises.loc[~(personas_paises['country'] == 'Brazil'),'country'] = 0

### 3: Features según el tiempo de cada sesión

   Función que itera sobre las columnas 'person', 'timestamp' y las sesiones de los usuarios 'new_vs_returning'. 
   
   El timestamp del df pasado como argumento debe estar ordenado de manera descendente.
  
   Si no se encuentra el logueo a la página (New o Returning), se marca esa sesión de tiempo 0 (Esto se marca cuando se pasa a otra persona y había eventos sin tener en cuenta el logueo del usuario) 
   
   Si la diferencia entre el tiempo de los eventos es mayor a un día, no se toma esa sesión.
   
  El tiempo de sesión es devuelto en un df junto al id de la persona y el timestamp del logueo 

In [17]:
def mostrar_tiempo(df):    
    dic_events_time = {}
    person = ""
    rows = []
    for index, row in df.iterrows():
        if(person == ""):
            person = row["person"]
            last_event_time = row['timestamp']
            continue            
        tiempo = ((last_event_time - row['timestamp']).total_seconds())
        if(tiempo > 86400):
            last_event_time = row['timestamp']
        if (row['new_vs_returning'] == 'New' or row['new_vs_returning'] == 'Returning' or person != row["person"]):
            if(row['person'] != person):
                tiempo = 0
            rows.append([person,tiempo,row['timestamp']])
            person = ""
    return pd.DataFrame(rows,columns=('person', 'time_event', 'timestamp'))

In [18]:
t0 = time()

tiempo_sesiones = mostrar_tiempo(datos[['person','timestamp','new_vs_returning']].\
                                                            sort_values(by=['person','timestamp'],ascending=False))

tf = (time() - t0)/60
print ("Tiempo de ejecución: %0.5f minutes." % tf)

Tiempo de ejecución: 7.17078 minutes.


In [19]:
tiempo_sesiones.sort_values(by='time_event',ascending=False).head(5)

,person,time_event,timestamp
4358,f16f4ae3,12618905.0,2018-01-02 23:50:27
169,ff69ad22,11592300.0,2018-01-16 20:14:21
63334,2c18b4db,11386282.0,2018-01-12 00:59:05
5711,ecb947e4,11056671.0,2018-01-11 22:23:16
13324,d3a2e324,10897170.0,2018-01-23 18:34:06


Se agrega el tiempo de la sesion como time_event en el df datos

In [20]:
datos = pd.merge(datos,tiempo_sesiones,how='left',on=['person','timestamp']).fillna(0)

#### Máximo tiempo de una sesión por persona y quincena

In [21]:
max_tiempo_q = datos[['person','quincena','time_event']]
max_tiempo_q = max_tiempo_q.groupby(['person','quincena']).agg({'time_event':'max'})
max_tiempo_q = max_tiempo_q.unstack()
max_tiempo_q.columns = max_tiempo_q.columns.droplevel(0)
max_tiempo_q = max_tiempo_q.reset_index(level=0,drop=False)
max_tiempo_q.fillna(0,inplace=True)

In [22]:
j = 0
for i in max_tiempo_q.columns:
    max_tiempo_q.rename(columns={j:'max_tiempo_sesion_q_'+str(j)},inplace=True)
    j =  j + 1

Usuarios con máximo tiempo de una sesión en la última quincena

In [23]:
max_tiempo_q['max_tiempo_sesion_q_9'].sort_values(ascending = False).head()

1729     1295779.0
22212    1192904.0
31431    1128788.0
29132    1065523.0
12113    1023401.0
Name: max_tiempo_sesion_q_9, dtype: float64

#### Promedio tiempo de sesión por persona y quincena

In [24]:
promedio_tiempo_q = datos[['person','quincena','time_event']]
promedio_tiempo_q = promedio_tiempo_q.groupby(['person','quincena']).agg({'time_event':'mean'})
promedio_tiempo_q = promedio_tiempo_q.unstack()
promedio_tiempo_q.columns = promedio_tiempo_q.columns.droplevel(0)
promedio_tiempo_q = promedio_tiempo_q.reset_index(level=0,drop=False)
promedio_tiempo_q.fillna(0,inplace=True)
j = 0
for i in promedio_tiempo_q.columns:
    promedio_tiempo_q.rename(columns={j:'prom_tiempo_sesion_q_'+str(j)},inplace=True)
    j =  j + 1

#### Usuarios con mayor promedio en el tiempo de sesión de la última quincena

In [25]:
promedio_tiempo_q['prom_tiempo_sesion_q_9'].sort_values(ascending = False).head()

31431    376262.666667
13181    350251.000000
17342    283049.454545
7145     266458.400000
1729     259155.800000
Name: prom_tiempo_sesion_q_9, dtype: float64

### 4. Dias entre compras

Funcion que toma el gap entre compras desde registración hasta el 1ro de junio (en días)

In [26]:
def dist_compras(df):
    rows = []
    person = ""
    for index,row in df.iterrows():
        if(person != row['person']):
            if(person != ""):
                gap = (datetime(2018, 6, 1, 0, 0) - dia).total_seconds()
                rows.append([person,dia,gap])
            person = row['person']
            dia = row['timestamp']
        else:
            gap = (row['timestamp'] - dia).total_seconds()
            rows.append([person,dia,gap])
            dia = row['timestamp']
    return pd.DataFrame(rows,columns=('person','timestamp','distance'))

Se utilizan las compras y se les agrega, como timestamp, la fecha de registro de la persona.

In [27]:
compradores = datos[['person','timestamp','event']].sort_values(by=['person','timestamp'])
compradores = compradores.loc[compradores['event']=='conversion',['person','timestamp']]
sesion_comprar = compradores.append(usuariosyregistros)
sesion_comprar =sesion_comprar.reset_index(drop=True)
sesion_comprar.drop(columns='new_vs_returning',inplace=True)
sesion_comprar.sort_values(by=['person','timestamp'],inplace=True)

/home/mariano/ENTER/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [28]:
sesion_comprar['timestamp'] = pd.to_datetime(sesion_comprar['timestamp'])
distancias = dist_compras(sesion_comprar)

#### Se crean los features: 'hace_cuanto_no_convierte','max_sin_convertir','promedio_dias_conversion'

El primero es el tiempo desde que dejo de convertir, el segundo el máximo tiempo que estuvo sin convertir y el tercero el promedio de tiempo en cuál convierte cada persona.

In [29]:
hace_cuanto_no_convierte = distancias.sort_values(by='timestamp',ascending=False).\
                                            drop_duplicates('person').drop(columns='timestamp').\
                                                rename(columns={'distance':'tiempo_sin_convertir'})
max_sin_convertir = distancias[['person','distance']].groupby('person').max().reset_index().\
                                                    rename(columns={'distance':'max_sin_convertir'})
promedio_tiempo_conversion = distancias[['person','distance']].groupby('person').mean().reset_index().\
                                                    rename(columns={'distance':'convierte_cada_x_tiempo'})

##### Se agrega el tiempo desde que esta registrada la persona: 'registration_time'

In [30]:
personas = datos[['person','timestamp','new_vs_returning']]
personas = personas.loc[personas['new_vs_returning'] == 'New']
personas['registration_time'] = (datetime(2018, 6, 1, 0, 0) - personas['timestamp'])
personas['registration_time'] = personas['registration_time'].apply(lambda x: x.total_seconds())
personas.drop(columns=['new_vs_returning','timestamp'],inplace=True)
personas.drop_duplicates('person',inplace=True)

In [31]:
columnas_tiempos = pd.merge(personas,max_sin_convertir,how='left',on='person')
columnas_tiempos = pd.merge(columnas_tiempos,hace_cuanto_no_convierte,how='left',on='person')
columnas_tiempos = pd.merge(columnas_tiempos,promedio_tiempo_conversion,how='left',on='person')
columnas_tiempos.isnull().sum()

person                     0
registration_time          0
max_sin_convertir          3
tiempo_sin_convertir       3
convierte_cada_x_tiempo    3
dtype: int64

Les agrego su tiempo de registro como maximo/tiempo sin convertir

In [32]:
columnas_tiempos.loc[(columnas_tiempos['max_sin_convertir'].isnull()) | (columnas_tiempos['tiempo_sin_convertir'].\
    isnull()) | (columnas_tiempos['convierte_cada_x_tiempo'].isnull()),\
    ['max_sin_convertir','tiempo_sin_convertir','convierte_cada_x_tiempo']] = columnas_tiempos['registration_time']

In [33]:
columnas_tiempos.head()

,person,registration_time,max_sin_convertir,tiempo_sin_convertir,convierte_cada_x_tiempo
0,29ebb414,1207424.0,1207424.0,1207424.0,1207424.0
1,de8fe91b,1207437.0,1207437.0,1207437.0,1207437.0
2,f87be219,1206287.0,1206287.0,1206287.0,1206287.0
3,e2bfe05f,1205365.0,1205365.0,1205365.0,1205365.0
4,bb78c182,1205309.0,1205309.0,1205309.0,1205309.0


### 5. Celulares

In [34]:
sistema_celulares = datos.loc[datos['device_type']=='Smartphone',['person','timestamp','operating_system_version']]\
                                    .sort_values(by=['person','timestamp'],ascending=False).drop_duplicates('person')
sistema_celulares['operating_system_version'] = sistema_celulares.operating_system_version.str.split('.').str[0]

In [35]:
iOS = sistema_celulares.loc[sistema_celulares.operating_system_version.str.contains('iOS')]
iOS.operating_system_version.value_counts()

iOS 11    1111
iOS 10     407
iOS 9      153
iOS 7       46
iOS 8       10
iOS 6        2
Name: operating_system_version, dtype: int64

In [36]:
Android = sistema_celulares.loc[sistema_celulares.operating_system_version.str.contains('Android')]
Android.operating_system_version.value_counts()

Android 6     7448
Android 7     6049
Android 5     3367
Android 4     1596
Android 8      442
Android         27
Android 2        7
Android 3        2
Android 10       1
Name: operating_system_version, dtype: int64

Funcion en la que se indican los sistemas 'iOS 6', 'iOS7', 'iOS8','Android 2', 'Android 3' y 'Android' cómo 1 y los demás 0. Indicando que dichos celulares necesitan una renovación.

In [37]:
def esViejo(celular):
    if (celular == "iOS 6" or celular == "iOS 7" or celular == "Android 2" or celular == "Android 3"):
        return 1
    else:
        return 0

In [38]:
sistema_celulares["celularViejo"] = sistema_celulares["operating_system_version"].apply(lambda x: esViejo(x))
sistema_celulares = sistema_celulares[["person", "celularViejo"]]

### 6. Cantidad de sesiones

In [39]:
datos_sesiones_q = datos[['person','quincena','new_vs_returning']]
datos_sesiones_q.fillna(0,inplace=True)
datos_sesiones_q.loc[(datos_sesiones_q['new_vs_returning'] == 'New') |\
                                     (datos_sesiones_q['new_vs_returning'] == 'Returning'),'new_vs_returning'] = 1
datos_sesiones_q = datos_sesiones_q.groupby(['person','quincena']).agg({'new_vs_returning':'sum'})
datos_sesiones_q = datos_sesiones_q.unstack()
datos_sesiones_q.columns = datos_sesiones_q.columns.droplevel(0)
datos_sesiones_q = datos_sesiones_q.reset_index(level=0,drop=False)
datos_sesiones_q.fillna(0,inplace=True)

j = 0
for i in datos_sesiones_q.columns:
    datos_sesiones_q.rename(columns={j:'cant_sesiones_q_'+str(j)},inplace=True)
    j =  j + 1

/home/mariano/ENTER/lib/python3.6/site-packages/pandas/core/frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)
/home/mariano/ENTER/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


## Lectura de los csv (train y test) y mergeo del modelo.

In [41]:
modelo = pd.merge(eventosPersona,eventosPersonaQ,on='person')
modelo = pd.merge(modelo,max_tiempo_q,on='person')
modelo = pd.merge(modelo,promedio_tiempo_q,on='person')
modelo = pd.merge(modelo,columnas_tiempos,on='person')
modelo = pd.merge(modelo,sistema_celulares,on='person',how='left')
modelo = pd.merge(modelo,personas_paises,on='person')
modelo = pd.merge(modelo,datos_sesiones_q,on='person')
modelo.fillna(0,inplace=True)

In [42]:
train = pd.read_csv("labels_training_set.csv",low_memory=False)
test = pd.read_csv("trocafone_kaggle_test.csv",low_memory=False)

In [43]:
train = train.merge(modelo,how='left',on='person')
test =test.merge(modelo,how='left',on='person')

In [44]:
test.to_csv('test_final.csv', encoding='utf-8', index=False)
train.to_csv('train_final.csv', encoding='utf-8', index=False)